https://www.tensorflow.org/tutorials/layers

tf.layers documentation: https://www.tensorflow.org/api_docs/python/tf/layers

tf.layers.conv2: https://www.tensorflow.org/api_docs/python/tf/layers/conv2d

In [1]:
import numpy as np
import tensorflow as tf

In [ ]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN"""
    # Input layer 
    # input_layer = [batch_size, image_height, image_width, channels]
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    
    # Convolutional Layer 1 
    '''
    filters = number of [5,5] chunks taken from 
    the input grid and compared over it to 
    make a convolution 
    
    padding = same: output tensor should have 
    the same height and width as input tensor
    
    activation: activation function (ReLU)
    '''
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    
    # Pooling Layer 1 
    '''    
    inputs: the input tensor
        - requires shape [batch_size, image_height, image_width, channels]
        - the output from the first conv layer 
        - ours has shape [batch_size, 28, 28, 32]
    
    pool_size: max pooling filter [height, width]
    
    strides: size of stride
        - the subregions extracted by the filter 
        should be separated by 2 pixels in both 
        the height and width dimensions 
        - (for a 2x2 filter, this means that none of 
        the regions extracted will overlap)
        
    returns output tensor with shape [batch_size, 14, 14, 32]
        - the 2x2 filter reduced height and weight by 50% each
    '''
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    # Convolutional Layer 2 
    '''
    - takes the output tensor of pool1 as input 
    - output shape is [batch_size, 14, 14, 64]
    - width and height are same as pool1 beacuse of 
    padding="same"
    '''
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    
    # Pooling Layer 2 (same as pooling layer 1)
    '''
    - takes conv2 as input 
    - output pool2 has shape [batch_size, 7, 7, 64] 
        - (50% reduction of height and width from conv2)
    '''
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    # Dense Layer (aka fully connected layer)
    '''
    first flatten feature map (pool2) to shape
    [batch_size, features] so the tensor is only 2d
    
    - the -1 means batch_size dimension will be 
    dynamically calculated based on # of ex in input data 
    -  Each example has 7 (pool2 height) * 7 (pool2 width) * 64 (pool2 channels) features
    - output tensor has shape [batch_size, 3136]
    '''
    
    pool2_flat = tf.reshape(pool2, [-1, 7*7*64])
    
    '''
    - input: flattened feature map pool2_flat 
    - units: # of neurons in the dense layer (this seems arbitrary?)
    - activation: ReLU (same as in convolutional layers)
    '''
    
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    
    # dropout 
    '''
    improves the results of the model 
    
    - inputs: dense tensor 
    - rate: dropout rate; 0.4 == 40% of elements 
    will be randomly dropped out during training
    - training: bool 
        - dropout only run if True 
        - ours: check if th emodel is in TRAIN mode
    
    output tensor has shape [batch_size, 1024]
    '''

    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # Logits Layer 
    # returns the raw values for our predictions (classes 0-9)
    # uses linear activation (the default)
    # output tensor has shape [batch_size, 10]
    
    logits = tf.layers.dense(inputs=dropout, units=10)
    